In [1]:
# !pip install fhir.resources
import json
from fhir.resources.R4B.bundle import Bundle

In [4]:
json_file_path = 'synthea/output/fhir/Delbert384_Cummings51_39055503-68de-ca85-9d7e-2548201875ee.json'

In [5]:
with open(json_file_path) as f:
    data = json.load(f)

# Validate root bundle structure
bundle = Bundle(**data)

In [ ]:
def codeable_concept_parser(cc):
    code = ""

    for i,codings in enumerate(cc.coding):
        code = code + codings.code
        if i > 0 and i < len(cc.coding):
            code = code + ","
    return code

def n_codeable_concept_parser(codeable_list):
    for i,codeable in enumerate(codeable_list):
        codes = codeable_concept_parser(codeable)
        if i > 0 and i == len(codeable_list) - 1:
            codes += ","

    return codes

## Pataient Resource

In [6]:
type(bundle.entry[0].resource)

fhir.resources.R4B.patient.Patient

In [7]:
pat = bundle.entry[0].resource

In [8]:
patient_schema = dict()

In [9]:
pat.name

[HumanName(fhir_comments=None, extension=None, id=None, family='Cummings51', family__ext=None, given=['Delbert384', 'Bill567'], given__ext=None, period=None, prefix=['Mr.'], prefix__ext=None, suffix=None, suffix__ext=None, text=None, text__ext=None, use='official', use__ext=None)]

In [10]:
patient_schema['last_name'] = pat.name[0].family
patient_schema['prefix_name'] = pat.name[0].prefix[0]
patient_schema['first_name'] = pat.name[0].given[0]
patient_schema['middle_name'] = pat.name[0].given[1] if len(pat.name[0].given) > 1 else None

In [11]:
pat.identifier

[Identifier(fhir_comments=None, extension=None, id=None, assigner=None, period=None, system='https://github.com/synthetichealth/synthea', system__ext=None, type=None, use=None, use__ext=None, value='39055503-68de-ca85-9d7e-2548201875ee', value__ext=None),
 Identifier(fhir_comments=None, extension=None, id=None, assigner=None, period=None, system='http://hospital.smarthealthit.org', system__ext=None, type=CodeableConcept(fhir_comments=None, extension=None, id=None, coding=[Coding(fhir_comments=None, extension=None, id=None, code='MR', code__ext=None, display='Medical Record Number', display__ext=None, system='http://terminology.hl7.org/CodeSystem/v2-0203', system__ext=None, userSelected=None, userSelected__ext=None, version=None, version__ext=None)], text='Medical Record Number', text__ext=None), use=None, use__ext=None, value='39055503-68de-ca85-9d7e-2548201875ee', value__ext=None),
 Identifier(fhir_comments=None, extension=None, id=None, assigner=None, period=None, system='http://hl7.

In [12]:
patient_schema['ID'] = pat.id 
for identifier in pat.identifier:
    if identifier.type != None:
        patient_schema[identifier.type.coding[0].code] = identifier.value

In [13]:
for i,contacts in enumerate(pat.telecom):
    patient_schema[f'phone_{i}'] = contacts.value

In [14]:
patient_schema['gender'] = getattr(pat, 'gender', None)
patient_schema['deceased'] = getattr(pat, 'deceased', None)
patient_schema['deceased_boolean'] = pat.deceasedBoolean
patient_schema['deceased_timestamp'] = pat.deceasedDateTime
patient_schema['multiple_birth_boolean'] = pat.multipleBirthBoolean
patient_schema['multiple_birth_integer'] = pat.multipleBirthInteger
patient_schema['marital_status'] = (pat.maritalStatus.coding[0].display).lower()

In [15]:
patient_schema['city'] = pat.address[0].city
patient_schema['country'] = pat.address[0].country
patient_schema['postal_code'] = pat.address[0].postalCode
patient_schema['state'] = pat.address[0].state
patient_schema['dob'] = pat.birthDate

In [16]:
patient_schema

{'last_name': 'Cummings51',
 'prefix_name': 'Mr.',
 'first_name': 'Delbert384',
 'middle_name': 'Bill567',
 'ID': '39055503-68de-ca85-9d7e-2548201875ee',
 'MR': '39055503-68de-ca85-9d7e-2548201875ee',
 'SS': '999-49-5247',
 'DL': 'S99996490',
 'PPN': 'X37616068X',
 'phone_0': '555-424-2136',
 'gender': 'male',
 'deceased': None,
 'deceased_boolean': None,
 'deceased_timestamp': None,
 'multiple_birth_boolean': False,
 'multiple_birth_integer': None,
 'marital_status': 'married',
 'city': 'Duxbury',
 'country': 'US',
 'postal_code': '02332',
 'state': 'MA',
 'dob': datetime.date(1982, 8, 18)}

In [17]:
def patient_resource_parser(pat):
    patient_schema = dict()
    patient_schema['last_name'] = pat.name[0].family
    patient_schema['prefix_name'] = pat.name[0].prefix[0]
    patient_schema['first_name'] = pat.name[0].given[0]
    patient_schema['middle_name'] = pat.name[0].given[1] if len(pat.name[0].given) > 1 else None
    patient_schema['ID'] = pat.id 
    for identifier in pat.identifier:
        if identifier.type != None:
            patient_schema[identifier.type.coding[0].code] = identifier.value
    for i,contacts in enumerate(pat.telecom):
        patient_schema[f'phone_{i}'] = contacts.value
    patient_schema['gender'] = getattr(pat, 'gender', None)
    patient_schema['deceased'] = getattr(pat, 'deceased', None)
    patient_schema['deceased_boolean'] = pat.deceasedBoolean
    patient_schema['deceased_timestamp'] = pat.deceasedDateTime
    patient_schema['multiple_birth_boolean'] = pat.multipleBirthBoolean
    patient_schema['multiple_birth_integer'] = pat.multipleBirthInteger
    patient_schema['marital_status'] = (pat.maritalStatus.coding[0].display).lower()
    patient_schema['city'] = pat.address[0].city
    patient_schema['country'] = pat.address[0].country
    patient_schema['postal_code'] = pat.address[0].postalCode
    patient_schema['state'] = pat.address[0].state
    patient_schema['dob'] = pat.birthDate

    return patient_schema

In [18]:
patient_resource_parser(pat)

{'last_name': 'Cummings51',
 'prefix_name': 'Mr.',
 'first_name': 'Delbert384',
 'middle_name': 'Bill567',
 'ID': '39055503-68de-ca85-9d7e-2548201875ee',
 'MR': '39055503-68de-ca85-9d7e-2548201875ee',
 'SS': '999-49-5247',
 'DL': 'S99996490',
 'PPN': 'X37616068X',
 'phone_0': '555-424-2136',
 'gender': 'male',
 'deceased': None,
 'deceased_boolean': None,
 'deceased_timestamp': None,
 'multiple_birth_boolean': False,
 'multiple_birth_integer': None,
 'marital_status': 'married',
 'city': 'Duxbury',
 'country': 'US',
 'postal_code': '02332',
 'state': 'MA',
 'dob': datetime.date(1982, 8, 18)}

## Encounter Resource

In [256]:
type(bundle.entry[1].resource)

fhir.resources.R4B.encounter.Encounter

In [257]:
encounter = bundle.entry[1].resource

In [258]:
encounter_schema = dict()

In [259]:
type(encounter)

fhir.resources.R4B.encounter.Encounter

In [260]:
encounter_schema['ID'] = encounter.id

In [261]:
encounter_schema['location'] = encounter.location[0].location.display

In [262]:
encounter_schema['patient_id'] = encounter.subject.reference

In [263]:
encounter_schema['snomed_code'] = encounter.type[0].coding[0].code

In [264]:
encounter_schema['service_provider'] = encounter.serviceProvider.display

In [265]:
encounter_schema['reason_code'] = encounter.reasonCode[0].coding[0].code

TypeError: 'NoneType' object is not subscriptable

In [266]:
encounter_schema['end_timestamp'] = encounter.period.end
encounter_schema['start_timestamp'] = encounter.period.start

In [267]:
encounter_schema['doctor'] = encounter.participant[0].individual.display

In [268]:
def encounter_resource_parser(encounter):
    encounter_schema = dict()
    encounter_schema['ID'] = encounter.id
    encounter_schema['location'] = encounter.location[0].location.display
    encounter_schema['patient_id'] = encounter.subject.reference
    encounter_schema['snomed_code'] = encounter.type[0].coding[0].code
    encounter_schema['service_provider'] = encounter.serviceProvider.display
    if encounter.reasonCode == None:
        encounter_schema['reason_code'] = None
    else:
        encounter_schema['reason_code'] = encounter.reasonCode[0].coding[0].code
    encounter_schema['end_timestamp'] = encounter.period.end
    encounter_schema['start_timestamp'] = encounter.period.start
    encounter_schema['doctor'] = encounter.participant[0].individual.display

    return encounter_schema

In [125]:
parse_results = []
for entry in bundle.entry:
    if entry.request.url == 'Encounter':
        result = encounter_resource_parser(entry.resource)
        parse_results.append(result)

## Condition Resource

In [269]:
type(bundle.entry[2].resource)

fhir.resources.R4B.condition.Condition

In [270]:
condition = bundle.entry[2].resource

In [271]:
condition

Condition(fhir_comments=None, id='d0dbc708-18ea-9d78-1135-d77190d7220c', implicitRules=None, implicitRules__ext=None, language=None, language__ext=None, meta=Meta(fhir_comments=None, extension=None, id=None, lastUpdated=None, lastUpdated__ext=None, profile=['http://hl7.org/fhir/us/core/StructureDefinition/us-core-condition-encounter-diagnosis'], profile__ext=None, security=None, source=None, source__ext=None, tag=None, versionId=None, versionId__ext=None), contained=None, extension=None, modifierExtension=None, text=None, abatementAge=None, abatementDateTime=None, abatementDateTime__ext=None, abatementPeriod=None, abatementRange=None, abatementString=None, abatementString__ext=None, asserter=None, bodySite=None, category=[CodeableConcept(fhir_comments=None, extension=None, id=None, coding=[Coding(fhir_comments=None, extension=None, id=None, code='encounter-diagnosis', code__ext=None, display='Encounter Diagnosis', display__ext=None, system='http://terminology.hl7.org/CodeSystem/conditi

In [272]:
condition_schema = dict()

In [273]:
condition_schema['ID'] = condition.id

In [274]:
condition_schema['patient_id'] = condition.subject.reference

In [275]:
condition_schema['clinical_status'] = condition.clinicalStatus.coding[0].code

In [276]:
condition_schema['verification_status'] = condition.verificationStatus.coding[0].code

In [277]:
condition_schema['condition_category'] = condition.category[0].coding[0].code

In [278]:
condition_schema['snomed_code'] = condition.code.coding[0].code

In [279]:
if condition.bodySite == None:
    condition_schema['body_Site'] = None
else:
    condition_schema['body_Site'] = condition.bodySite.coding[0].code

In [280]:
if condition.severity == None:
    condition_schema['severity'] = None
else:
    condition_schema['severity'] = condition.severity.coding[0].code

In [281]:
condition_schema['onsetAge'] = getattr(condition, 'onsetAge', None)
condition_schema['onsetDateTime'] = getattr(condition, 'onsetDateTime', None)
condition_schema['abatementAge'] = getattr(condition, 'abatementAge', None)
condition_schema['abatementDateTime'] = getattr(condition, 'abatementDateTime', None)
condition_schema['recordedDate'] = getattr(condition, 'recordedDate', None)

In [282]:
condition_schema['stage'] = condition.stage

In [283]:
condition_schema['evidence'] = condition.evidence

In [284]:
def condition_resource_parser(condition):
    condition_schema = dict()
    if condition.id == None:
        return None
    condition_schema['ID'] = condition.id
    if condition.subject == None:
        return None
    condition_schema['patient_id'] = condition.subject.reference
    condition_schema['clinical_status'] = condition.clinicalStatus.coding[0].code
    condition_schema['verification_status'] = condition.verificationStatus.coding[0].code
    condition_schema['condition_category'] = condition.category[0].coding[0].code
    condition_schema['snomed_code'] = condition.code.coding[0].code
    if condition.bodySite == None:
        condition_schema['body_Site'] = None
    else:
        condition_schema['body_Site'] = condition.bodySite.coding[0].code
    if condition.severity == None:
        condition_schema['severity'] = None
    else:
        condition_schema['severity'] = condition.severity.coding[0].code
    condition_schema['onsetAge'] = getattr(condition, 'onsetAge', None)
    condition_schema['onsetDateTime'] = getattr(condition, 'onsetDateTime', None)
    condition_schema['abatementAge'] = getattr(condition, 'abatementAge', None)
    condition_schema['abatementDateTime'] = getattr(condition, 'abatementDateTime', None)
    condition_schema['recordedDate'] = getattr(condition, 'recordedDate', None)
    condition_schema['stage'] = condition.stage
    condition_schema['evidence'] = condition.evidence

    return condition_schema

In [285]:
parse_results = []
for entry in bundle.entry: ## TESTING
    if entry.request.url == 'Condition':
        result = condition_resource_parser(entry.resource)
        parse_results.append(result)

## Device Resource

In [149]:
type(bundle.entry[9].resource)

fhir.resources.R4B.device.Device

In [150]:
device = bundle.entry[9].resource
device

Device(fhir_comments=None, id='033e4431-ed01-313a-ad3c-dcd7bc18baca', implicitRules=None, implicitRules__ext=None, language=None, language__ext=None, meta=Meta(fhir_comments=None, extension=None, id=None, lastUpdated=None, lastUpdated__ext=None, profile=['http://hl7.org/fhir/us/core/StructureDefinition/us-core-implantable-device'], profile__ext=None, security=None, source=None, source__ext=None, tag=None, versionId=None, versionId__ext=None), contained=None, extension=None, modifierExtension=None, text=None, contact=None, definition=None, deviceName=[DeviceDeviceName(fhir_comments=None, extension=None, id=None, modifierExtension=None, name='Home nebulizer (physical object)', name__ext=None, type='user-friendly-name', type__ext=None)], distinctIdentifier='33093132134901', distinctIdentifier__ext=None, expirationDate=datetime.datetime(2029, 6, 12, 20, 43, 8, tzinfo=TzInfo(UTC)), expirationDate__ext=None, identifier=None, location=None, lotNumber='2319', lotNumber__ext=None, manufactureDa

In [151]:
device.id

'033e4431-ed01-313a-ad3c-dcd7bc18baca'

In [152]:
device.deviceName[0].name

'Home nebulizer (physical object)'

In [153]:
device.deviceName[0].type

'user-friendly-name'

In [154]:
device.contact

In [155]:
device.definition

In [156]:
device.location

In [157]:
device.lotNumber

'2319'

In [158]:
device.distinctIdentifier

'33093132134901'

In [159]:
device.expirationDate

datetime.datetime(2029, 6, 12, 20, 43, 8, tzinfo=TzInfo(UTC))

In [160]:
device.manufactureDate

datetime.datetime(2004, 5, 28, 20, 43, 8, tzinfo=TzInfo(UTC))

In [161]:
device.manufacturer

In [162]:
device.type

CodeableConcept(fhir_comments=None, extension=None, id=None, coding=[Coding(fhir_comments=None, extension=None, id=None, code='170615005', code__ext=None, display='Home nebulizer (physical object)', display__ext=None, system='http://snomed.info/sct', system__ext=None, userSelected=None, userSelected__ext=None, version=None, version__ext=None)], text='Home nebulizer (physical object)', text__ext=None)

In [163]:
device.patient.reference

'urn:uuid:39055503-68de-ca85-9d7e-2548201875ee'

In [164]:
device.serialNumber

'5731404453565498925'

In [165]:
device.status

'active'

In [166]:
device.udiCarrier[0].carrierHRF

'(01)33093132134901(11)040528(17)290612(10)2319(21)5731404453565498925'

In [167]:
def device_resource_parser(device):
    device_schema = dict()
    if device.id == None:
        return None
    device_schema['ID'] = device.id
    device_schema['device_name'] = device.deviceName[0].name
    device_schema['device_type'] = device.deviceName[0].type
    device_schema['lot_number'] = getattr(device, 'lotNumber', None)
    device_schema['definition'] = getattr(device, 'definition', None)
    device_schema['location'] = getattr(device, 'location', None)
    device_schema['distinctIdentifier'] = getattr(device, 'distinctIdentifier', None)
    device_schema['expirationDate'] = getattr(device, 'expirationDate', None)
    device_schema['manufactureDate'] = getattr(device, 'manufactureDate', None)
    device_schema['manufacturer'] = getattr(device, 'manufacturer', None)
    device_schema['serialNumber'] = getattr(device, 'serialNumber', None)
    device_schema['status'] = getattr(device, 'status', None)
    device_schema['udi_id'] = device.udiCarrier[0].carrierHRF
    device_schema['patient_id'] = device.patient.reference

    return device_schema

In [168]:
parse_results = []
for entry in bundle.entry: ## TESTING
    if entry.request.url == 'Device':
        result = device_resource_parser(entry.resource)
        parse_results.append(result)

## Diagonostic Report

In [170]:
type(bundle.entry[3].resource)

fhir.resources.R4B.diagnosticreport.DiagnosticReport

In [171]:
report = bundle.entry[3].resource
report

DiagnosticReport(fhir_comments=None, id='6bc5bc5b-5644-116f-5b11-1999888da30c', implicitRules=None, implicitRules__ext=None, language=None, language__ext=None, meta=Meta(fhir_comments=None, extension=None, id=None, lastUpdated=None, lastUpdated__ext=None, profile=['http://hl7.org/fhir/us/core/StructureDefinition/us-core-diagnosticreport-note'], profile__ext=None, security=None, source=None, source__ext=None, tag=None, versionId=None, versionId__ext=None), contained=None, extension=None, modifierExtension=None, text=None, basedOn=None, category=[CodeableConcept(fhir_comments=None, extension=None, id=None, coding=[Coding(fhir_comments=None, extension=None, id=None, code='34117-2', code__ext=None, display='History and physical note', display__ext=None, system='http://loinc.org', system__ext=None, userSelected=None, userSelected__ext=None, version=None, version__ext=None), Coding(fhir_comments=None, extension=None, id=None, code='51847-2', code__ext=None, display='Evaluation + Plan note', 

In [172]:
report.id

'6bc5bc5b-5644-116f-5b11-1999888da30c'

In [173]:
report.category

[CodeableConcept(fhir_comments=None, extension=None, id=None, coding=[Coding(fhir_comments=None, extension=None, id=None, code='34117-2', code__ext=None, display='History and physical note', display__ext=None, system='http://loinc.org', system__ext=None, userSelected=None, userSelected__ext=None, version=None, version__ext=None), Coding(fhir_comments=None, extension=None, id=None, code='51847-2', code__ext=None, display='Evaluation + Plan note', display__ext=None, system='http://loinc.org', system__ext=None, userSelected=None, userSelected__ext=None, version=None, version__ext=None)], text=None, text__ext=None)]

In [174]:
report.code.coding[1]

Coding(fhir_comments=None, extension=None, id=None, code='51847-2', code__ext=None, display='Evaluation + Plan note', display__ext=None, system='http://loinc.org', system__ext=None, userSelected=None, userSelected__ext=None, version=None, version__ext=None)

In [175]:
report.effectiveDateTime

datetime.datetime(2000, 10, 11, 1, 43, 8, tzinfo=TzInfo(UTC))

In [176]:
report.encounter.reference

'urn:uuid:d4f80e47-91cc-d21d-94a1-767692fea519'

In [177]:
report.performer[0].display

'Dr. Regenia619 Bosco882'

In [178]:
report.subject.reference

'urn:uuid:39055503-68de-ca85-9d7e-2548201875ee'

In [179]:
def diagonistic_report_resource_parser(report):
    diagonistic_report_schema = dict()
    diagonistic_report_schema['ID'] = report.id
    diagonistic_report_schema['patient_id'] = report.subject.reference
    if report.performer != None:
        for i,doctor in enumerate(report.performer):
            diagonistic_report_schema[f'diagnosis_doctor_{i+1}'] = doctor.display
    for i,code_section in enumerate(report.code.coding):
        diagonistic_report_schema[f'diagnosis_loinc_code_{i+1}'] = code_section.code
    return diagonistic_report_schema

In [180]:
diagonistic_report_resource_parser(report)

{'ID': '6bc5bc5b-5644-116f-5b11-1999888da30c',
 'patient_id': 'urn:uuid:39055503-68de-ca85-9d7e-2548201875ee',
 'diagnosis_doctor_1': 'Dr. Regenia619 Bosco882',
 'diagnosis_loinc_code_1': '34117-2',
 'diagnosis_loinc_code_2': '51847-2'}

In [81]:
parse_results = []
for i,entry in enumerate(bundle.entry): ## TESTING
    if entry.request.url == 'DiagnosticReport':
        result = diagonistic_report_resource_parser(entry.resource)
        parse_results.append(result)

## Document Refrence

In [186]:
type(bundle.entry[11].resource)

fhir.resources.R4B.documentreference.DocumentReference

In [187]:
document_ref = bundle.entry[11].resource
document_ref

DocumentReference(fhir_comments=None, id='de34694e-fe86-5e46-27c6-063e49e39c79', implicitRules=None, implicitRules__ext=None, language=None, language__ext=None, meta=Meta(fhir_comments=None, extension=None, id=None, lastUpdated=None, lastUpdated__ext=None, profile=['http://hl7.org/fhir/us/core/StructureDefinition/us-core-documentreference'], profile__ext=None, security=None, source=None, source__ext=None, tag=None, versionId=None, versionId__ext=None), contained=None, extension=None, modifierExtension=None, text=None, authenticator=None, author=[Reference(fhir_comments=None, extension=None, id=None, display='Dr. Crystle668 McCullough561', display__ext=None, identifier=None, reference='Practitioner?identifier=http://hl7.org/fhir/sid/us-npi|9999998997', reference__ext=None, type=None, type__ext=None)], category=[CodeableConcept(fhir_comments=None, extension=None, id=None, coding=[Coding(fhir_comments=None, extension=None, id=None, code='clinical-note', code__ext=None, display='Clinical N

In [188]:
document_ref.id

'de34694e-fe86-5e46-27c6-063e49e39c79'

In [189]:
document_ref.status

'superseded'

In [190]:
document_ref.author[0].display

'Dr. Crystle668 McCullough561'

In [191]:
document_ref.category

[CodeableConcept(fhir_comments=None, extension=None, id=None, coding=[Coding(fhir_comments=None, extension=None, id=None, code='clinical-note', code__ext=None, display='Clinical Note', display__ext=None, system='http://hl7.org/fhir/us/core/CodeSystem/us-core-documentreference-category', system__ext=None, userSelected=None, userSelected__ext=None, version=None, version__ext=None)], text=None, text__ext=None)]

In [192]:
document_ref.docStatus

In [193]:
document_ref.content[0].attachment.data

b'\n2004-06-18\n\n# Chief Complaint\nNo complaints.\n\n# History of Present Illness\nDelbert384\n is a 21 year-old nonhispanic black male.\n\n# Social History\nPatient is married. Patient has never smoked.\n Patient identifies as heterosexual.\n\nPatient comes from a high socioeconomic background.\n Patient is a college graduate.\nPatient currently has Blue Cross Blue Shield.\n\n# Allergies\nNo Known Allergies.\n\n# Medications\nNo Active Medications.\n\n# Assessment and Plan\nPatient is presenting with chronic sinusitis (disorder). \n\n## Plan\n\n'

In [194]:
document_ref.context

DocumentReferenceContext(fhir_comments=None, extension=None, id=None, modifierExtension=None, encounter=[Reference(fhir_comments=None, extension=None, id=None, display=None, display__ext=None, identifier=None, reference='urn:uuid:548b2aa5-5d6a-c44a-a865-4dc46e5d84b4', reference__ext=None, type=None, type__ext=None)], event=None, facilityType=None, period=Period(fhir_comments=None, extension=None, id=None, end=datetime.datetime(2004, 6, 18, 20, 58, 8, tzinfo=TzInfo(UTC)), end__ext=None, start=datetime.datetime(2004, 6, 18, 20, 43, 8, tzinfo=TzInfo(UTC)), start__ext=None), practiceSetting=None, related=None, sourcePatientInfo=None)

In [195]:
document_ref.type.coding[0].code

'34117-2'

In [196]:
document_ref.subject.reference

'urn:uuid:39055503-68de-ca85-9d7e-2548201875ee'

In [197]:
document_ref.identifier

[Identifier(fhir_comments=None, extension=None, id=None, assigner=None, period=None, system='urn:ietf:rfc:3986', system__ext=None, type=None, use=None, use__ext=None, value='urn:uuid:52c4dd76-5e54-11ae-f0ae-491e249d43e1', value__ext=None)]

In [198]:
document_ref.custodian.display

'BETH ISRAEL DEACONESS HOSPITAL PLYMOUTH INC'

In [199]:
document_ref.date

datetime.datetime(2004, 6, 18, 20, 43, 8, 282000, tzinfo=TzInfo(UTC))

In [200]:
def document_refrence_resource_parser(document_ref):
    document_ref_schema = dict()
    if document_ref.id == None:
        return None
    document_ref_schema['ID'] = document_ref.id
    document_ref_schema['status'] = document_ref.status
    document_ref_schema['patient_id'] = document_ref.subject.reference
    document_ref_schema['date'] = document_ref.date
    document_ref_schema['maintained_by'] = document_ref.custodian.display
    for i, doctor in enumerate(document_ref.author):
        document_ref_schema[f'doctor_name_{i+1}'] = doctor.display
    for i, coding in  enumerate(document_ref.type.coding):
        document_ref_schema[f'document_type_code_{i+1}'] = coding.code ## LOINC Code
    for i, category in enumerate(document_ref.category):
        document_ref_schema[f'category_code_{i+1}'] = ','.join([coding.code for coding in category.coding])
    for i,doc in enumerate(document_ref.content):
        document_ref_schema[f'data_{i+1}'] = doc.attachment.data ## LOINC Code
    for i, encounter in enumerate(document_ref.context.encounter):
        document_ref_schema[f'encounter_id_{i+1}'] = encounter.reference
    
    return document_ref_schema

In [201]:
parse_results = []
for i,entry in enumerate(bundle.entry): ## TESTING
    if entry.request.url == 'DocumentReference':
        result = document_refrence_resource_parser(entry.resource)
        parse_results.append(result)

## Claim Resource

In [231]:
type(bundle.entry[5].resource)

fhir.resources.R4B.claim.Claim

In [232]:
claim = bundle.entry[5].resource
claim

Claim(fhir_comments=None, id='7e324630-dd87-d5d5-0372-94ae2dc5cf61', implicitRules=None, implicitRules__ext=None, language=None, language__ext=None, meta=None, contained=None, extension=None, modifierExtension=None, text=None, accident=None, billablePeriod=Period(fhir_comments=None, extension=None, id=None, end=datetime.datetime(2000, 10, 11, 2, 15, 48, tzinfo=TzInfo(UTC)), end__ext=None, start=datetime.datetime(2000, 10, 11, 1, 43, 8, tzinfo=TzInfo(UTC)), start__ext=None), careTeam=None, created=datetime.datetime(2000, 10, 11, 2, 15, 48, tzinfo=TzInfo(UTC)), created__ext=None, diagnosis=[ClaimDiagnosis(fhir_comments=None, extension=None, id=None, modifierExtension=None, diagnosisCodeableConcept=None, diagnosisReference=Reference(fhir_comments=None, extension=None, id=None, display=None, display__ext=None, identifier=None, reference='urn:uuid:d0dbc708-18ea-9d78-1135-d77190d7220c', reference__ext=None, type=None, type__ext=None), onAdmission=None, packageCode=None, sequence=1, sequence_

In [233]:
claim.id

'7e324630-dd87-d5d5-0372-94ae2dc5cf61'

In [234]:
claim.status

'active'

In [235]:
claim.type

CodeableConcept(fhir_comments=None, extension=None, id=None, coding=[Coding(fhir_comments=None, extension=None, id=None, code='professional', code__ext=None, display=None, display__ext=None, system='http://terminology.hl7.org/CodeSystem/claim-type', system__ext=None, userSelected=None, userSelected__ext=None, version=None, version__ext=None)], text=None, text__ext=None)

In [236]:
claim.use

'claim'

In [237]:
claim.patient.reference

'urn:uuid:39055503-68de-ca85-9d7e-2548201875ee'

In [238]:
claim.billablePeriod.end

datetime.datetime(2000, 10, 11, 2, 15, 48, tzinfo=TzInfo(UTC))

In [239]:
claim.created

datetime.datetime(2000, 10, 11, 2, 15, 48, tzinfo=TzInfo(UTC))

In [240]:
claim.diagnosis

[ClaimDiagnosis(fhir_comments=None, extension=None, id=None, modifierExtension=None, diagnosisCodeableConcept=None, diagnosisReference=Reference(fhir_comments=None, extension=None, id=None, display=None, display__ext=None, identifier=None, reference='urn:uuid:d0dbc708-18ea-9d78-1135-d77190d7220c', reference__ext=None, type=None, type__ext=None), onAdmission=None, packageCode=None, sequence=1, sequence__ext=None, type=None)]

In [241]:
claim.enterer

In [242]:
claim.insurer

In [243]:
claim.provider

Reference(fhir_comments=None, extension=None, id=None, display='UNITED MEDICAL CARE LLC', display__ext=None, identifier=None, reference='Organization?identifier=https://github.com/synthetichealth/synthea|4e56c7ec-99e5-3023-8e4f-95ad18a03f06', reference__ext=None, type=None, type__ext=None)

In [244]:
claim.priority.coding[0].code

'normal'

In [245]:
claim.prescription

In [246]:
claim.originalPrescription

In [247]:
claim.related

In [248]:
claim.payee

In [249]:
claim.referral

In [250]:
claim.facility.display

'UNITED MEDICAL CARE LLC'

In [251]:
claim.diagnosis[0].diagnosisReference.reference

'urn:uuid:d0dbc708-18ea-9d78-1135-d77190d7220c'

In [252]:
claim.insurance[0].coverage.display

'Blue Cross Blue Shield'

In [253]:
claim.item

[ClaimItem(fhir_comments=None, extension=None, id=None, modifierExtension=None, bodySite=None, careTeamSequence=None, careTeamSequence__ext=None, category=None, detail=None, diagnosisSequence=None, diagnosisSequence__ext=None, encounter=[Reference(fhir_comments=None, extension=None, id=None, display=None, display__ext=None, identifier=None, reference='urn:uuid:d4f80e47-91cc-d21d-94a1-767692fea519', reference__ext=None, type=None, type__ext=None)], factor=None, factor__ext=None, informationSequence=None, informationSequence__ext=None, locationAddress=None, locationCodeableConcept=None, locationReference=None, modifier=None, net=None, procedureSequence=None, procedureSequence__ext=None, productOrService=CodeableConcept(fhir_comments=None, extension=None, id=None, coding=[Coding(fhir_comments=None, extension=None, id=None, code='162673000', code__ext=None, display='General examination of patient (procedure)', display__ext=None, system='http://snomed.info/sct', system__ext=None, userSelect

In [254]:
claim.total.value

Decimal('704.200000000000045474735088646411895751953125')

In [255]:
def claim_resource_parser(claim):
    if claim.id == None:
        return None
    claim_schema = dict()
    claim_schema['ID'] = claim.id
    claim_schema['status'] = getattr(claim, 'status', None)
    claim_schema['claim_type_codes'] = ','.join([codings.code for codings in claim.type.coding])
    claim_schema['claim_use'] = getattr(claim, 'use', None)
    claim_schema['created'] = getattr(claim, 'created', None)
    claim_schema['enterer'] = getattr(claim, 'enterer', None)
    claim_schema['insurer'] = getattr(claim, 'insurer', None)
    claim_schema['provider'] = getattr(claim, 'provider', None)
    claim_schema['created'] = getattr(claim, 'created', None)
    claim_schema['prescription'] = getattr(claim, 'prescription', None)
    claim_schema['related'] = getattr(claim, 'related', None)
    claim_schema['referral'] = getattr(claim, 'referral', None)
    claim_schema['payee'] = getattr(claim, 'payee', None)
    claim_schema['procedure'] = getattr(claim, 'procedure', None)
    claim_schema['originalPrescription'] = getattr(claim, 'originalPrescription', None)
    claim_schema['careTeam'] = getattr(claim, 'careTeam', None)
    claim_schema['careTeam'] = getattr(claim, 'careTeam', None)
    claim_schema['accident'] = getattr(claim, 'accident', None)
    claim_schema['patient_id'] = claim.patient.reference
    claim_schema['billable_start'] = claim.billablePeriod.start
    claim_schema['billable_end'] = claim.billablePeriod.end
    claim_schema['claim_priority_code'] = claim.priority.coding[0].code
    if claim.facility != None:
        claim_schema['facility'] = claim.facility.display
    else:
        claim_schema['facility'] = None
    if claim.diagnosis != None:
        claim_schema['diagnosis_refrence_id'] = claim.diagnosis[0].diagnosisReference.reference
    else:
        claim_schema['diagnosis_refrence_id'] = None
    for i, insurance in enumerate(claim.insurance):
        claim_schema[f'insurance_{i+1}'] = insurance.coverage.display
    claim_schema['claim_amount'] = claim.total.value

    return claim_schema

In [228]:
parse_results = []
for i,entry in enumerate(bundle.entry): ## TESTING
    if entry.request.url == 'Claim':
        result = claim_resource_parser(entry.resource)
        parse_results.append(result)

## 

In [287]:
type(bundle.entry[6].resource)

fhir.resources.R4B.explanationofbenefit.ExplanationOfBenefit

In [289]:
explain_ob = bundle.entry[6].resource
explain_ob

ExplanationOfBenefit(fhir_comments=None, id='c8acfdaa-1fd6-da08-ee0b-5345de0923d2', implicitRules=None, implicitRules__ext=None, language=None, language__ext=None, meta=None, contained=[ServiceRequest(fhir_comments=None, id='referral', implicitRules=None, implicitRules__ext=None, language=None, language__ext=None, meta=None, contained=None, extension=None, modifierExtension=None, text=None, asNeededBoolean=None, asNeededBoolean__ext=None, asNeededCodeableConcept=None, authoredOn=None, authoredOn__ext=None, basedOn=None, bodySite=None, category=None, code=None, doNotPerform=None, doNotPerform__ext=None, encounter=None, identifier=None, instantiatesCanonical=None, instantiatesCanonical__ext=None, instantiatesUri=None, instantiatesUri__ext=None, insurance=None, intent='order', intent__ext=None, locationCode=None, locationReference=None, note=None, occurrenceDateTime=None, occurrenceDateTime__ext=None, occurrencePeriod=None, occurrenceTiming=None, orderDetail=None, patientInstruction=None,

In [290]:
explain_ob.id

'c8acfdaa-1fd6-da08-ee0b-5345de0923d2'

In [291]:
explain_ob.status

'active'

In [292]:
explain_ob.type

CodeableConcept(fhir_comments=None, extension=None, id=None, coding=[Coding(fhir_comments=None, extension=None, id=None, code='professional', code__ext=None, display=None, display__ext=None, system='http://terminology.hl7.org/CodeSystem/claim-type', system__ext=None, userSelected=None, userSelected__ext=None, version=None, version__ext=None)], text=None, text__ext=None)

In [293]:
explain_ob.subType

In [294]:
explain_ob.use

'claim'

In [339]:
explain_ob.patient.reference

'urn:uuid:39055503-68de-ca85-9d7e-2548201875ee'

In [367]:
explain_ob.careTeam

[ExplanationOfBenefitCareTeam(fhir_comments=None, extension=None, id=None, modifierExtension=None, provider=Reference(fhir_comments=None, extension=None, id=None, display=None, display__ext=None, identifier=None, reference='Practitioner?identifier=http://hl7.org/fhir/sid/us-npi|9999942599', reference__ext=None, type=None, type__ext=None), qualification=None, responsible=None, responsible__ext=None, role=CodeableConcept(fhir_comments=None, extension=None, id=None, coding=[Coding(fhir_comments=None, extension=None, id=None, code='primary', code__ext=None, display='Primary provider', display__ext=None, system='http://terminology.hl7.org/CodeSystem/claimcareteamrole', system__ext=None, userSelected=None, userSelected__ext=None, version=None, version__ext=None)], text=None, text__ext=None), sequence=1, sequence__ext=None)]

In [334]:
explain_ob.claim

Reference(fhir_comments=None, extension=None, id=None, display=None, display__ext=None, identifier=None, reference='urn:uuid:7e324630-dd87-d5d5-0372-94ae2dc5cf61', reference__ext=None, type=None, type__ext=None)

In [336]:
explain_ob.created

datetime.datetime(2000, 10, 11, 2, 15, 48, tzinfo=TzInfo(UTC))

In [361]:
explain_ob.claim.reference

'urn:uuid:7e324630-dd87-d5d5-0372-94ae2dc5cf61'

In [338]:
explain_ob.use

'claim'

In [341]:
explain_ob.billablePeriod.start

datetime.datetime(2000, 10, 11, 2, 15, 48, tzinfo=TzInfo(UTC))

In [347]:
explain_ob.insurer.display

'Blue Cross Blue Shield'

In [350]:
explain_ob.provider.reference

'Practitioner?identifier=http://hl7.org/fhir/sid/us-npi|9999942599'

In [378]:
explain_ob.diagnosis

[ExplanationOfBenefitDiagnosis(fhir_comments=None, extension=None, id=None, modifierExtension=None, diagnosisCodeableConcept=None, diagnosisReference=Reference(fhir_comments=None, extension=None, id=None, display=None, display__ext=None, identifier=None, reference='urn:uuid:d0dbc708-18ea-9d78-1135-d77190d7220c', reference__ext=None, type=None, type__ext=None), onAdmission=None, packageCode=None, sequence=1, sequence__ext=None, type=[CodeableConcept(fhir_comments=None, extension=None, id=None, coding=[Coding(fhir_comments=None, extension=None, id=None, code='principal', code__ext=None, display=None, display__ext=None, system='http://terminology.hl7.org/CodeSystem/ex-diagnosistype', system__ext=None, userSelected=None, userSelected__ext=None, version=None, version__ext=None)], text=None, text__ext=None)])]

In [379]:
explain_ob.diagnosis[0].diagnosisReference.reference

'urn:uuid:d0dbc708-18ea-9d78-1135-d77190d7220c'

In [380]:
for careteam in explain_ob.careTeam:
    print(careteam.sequence)
    print(careteam.provider.reference)

1
Practitioner?identifier=http://hl7.org/fhir/sid/us-npi|9999942599


In [382]:
explain_ob.procedure

In [387]:
explain_ob.insurance[0].coverage.display

'Blue Cross Blue Shield'

In [388]:
explain_ob.accident

In [389]:
explain_ob.item

[ExplanationOfBenefitItem(fhir_comments=None, extension=None, id=None, modifierExtension=None, adjudication=None, bodySite=None, careTeamSequence=None, careTeamSequence__ext=None, category=CodeableConcept(fhir_comments=None, extension=None, id=None, coding=[Coding(fhir_comments=None, extension=None, id=None, code='1', code__ext=None, display='Medical care', display__ext=None, system='https://bluebutton.cms.gov/resources/variables/line_cms_type_srvc_cd', system__ext=None, userSelected=None, userSelected__ext=None, version=None, version__ext=None)], text=None, text__ext=None), detail=None, diagnosisSequence=None, diagnosisSequence__ext=None, encounter=[Reference(fhir_comments=None, extension=None, id=None, display=None, display__ext=None, identifier=None, reference='urn:uuid:d4f80e47-91cc-d21d-94a1-767692fea519', reference__ext=None, type=None, type__ext=None)], factor=None, factor__ext=None, informationSequence=None, informationSequence__ext=None, locationAddress=None, locationCodeableC

In [390]:
explain_ob.total

[ExplanationOfBenefitTotal(fhir_comments=None, extension=None, id=None, modifierExtension=None, amount=Money(fhir_comments=None, extension=None, id=None, currency='USD', currency__ext=None, value=Decimal('704.200000000000045474735088646411895751953125'), value__ext=None), category=CodeableConcept(fhir_comments=None, extension=None, id=None, coding=[Coding(fhir_comments=None, extension=None, id=None, code='submitted', code__ext=None, display='Submitted Amount', display__ext=None, system='http://terminology.hl7.org/CodeSystem/adjudication', system__ext=None, userSelected=None, userSelected__ext=None, version=None, version__ext=None)], text='Submitted Amount', text__ext=None))]

In [392]:
def explaination_ob_resource_parser(explain_ob):
    if explain_ob.id == None:
        return None
    explain_ob_schema = dict()
    explain_ob_schema['ID'] = explain_ob.id
    explain_ob_schema['status'] = getattr(explain_ob, 'status', None)
    explain_ob_schema['type_code'] = codeable_concept_parser(explain_ob.type)
    explain_ob_schema['use'] = getattr(explain_ob, 'use', None)
    explain_ob_schema['patient_id'] = getattr(getattr(explain_ob, 'patient', None), 'refrence', None)
    explain_ob_schema['billable_start_date'] = getattr(getattr(explain_ob, 'billablePeriod', None), 'start', None)
    explain_ob_schema['billable_end_date'] = getattr(getattr(explain_ob, 'billablePeriod', None), 'start', None)
    explain_ob_schema['created'] = getattr(explain_ob,'created', None)
    explain_ob_schema['insurer'] = getattr(getattr(explain_ob, 'insurer', None), 'display', None)
    explain_ob_schema['provider'] = getattr(getattr(explain_ob, 'provider', None), 'reference', None)
    explain_ob_schema['outcome'] = getattr(explain_ob, 'outcome', None)
    explain_ob_schema['claim_id'] = getattr(getattr(explain_ob, 'claim', None), 'reference', None)
    explain_ob_schema['procedure'] = explain_ob.procedure
    for careteam in explain_ob.careTeam:
        explain_ob_schema[f'careteam_{careteam.sequence}_reference_id'] = careteam.provider.reference
    for diagnosis in explain_ob.diagnosis:
        explain_ob_schema[f'diagnosis_{diagnosis.sequence}_reference_id'] = diagnosis.diagnosisReference.reference
    for i,insurance in enumerate(explain_ob.insurance):
        explain_ob_schema[f'insurance_{i+1}'] = insurance.coverage.display


    return explain_ob_schema

In [325]:
def codeable_concept_parser(cc):
    code = ""

    for i,codings in enumerate(cc.coding):
        code = code + codings.code
        if i > 0 and i < len(cc.coding):
            code = code + ","
    return code

In [ ]:
def observation_resource_parser(obs):
    if obs.id == None:
        return None
    observation_schema = dict()
    observation_schema['ID'] = obs.id
    observation_schema['based_on'] = getattr(obs,'basedOn', None)
    observation_schema['part_of'] = getattr(obs,'partOf', None)
    observation_schema['status'] = getattr(obs,'status', None)
    observation_schema['category_type'] = n_codeable_concept_parser(obs.category)
    observation_schema['loinc_code'] = codeable_concept_parser(obs.code)
    observation_schema['patient_id'] = getattr(getattr(obs,'subject',None),'reference',None)
    observation_schema['focus'] = getattr(obs, 'focus', None)
    observation_schema['encounter_id'] = getattr(getattr(obs,'encounter',None),'reference',None)
    observation_schema['effective_datetime'] = getattr(obs, 'effectiveDateTime', None)
    observation_schema['effective_period'] = getattr(obs, 'effectivePeriod', None)
    observation_schema['effective_timing'] = getattr(obs, 'effectiveTiming', None)
    observation_schema['issued'] = getattr(obs, 'issued', None)
    observation_schema['performer'] = getattr(obs, 'performer', None)
    observation_schema['value_boolean'] = getattr(obs, 'valueBoolean', None)
    observation_schema['value_codeable_concept'] = getattr(obs, 'valueCodeableConcept', None)
    observation_schema['value_datetime'] = getattr(obs, 'valueDateTime', None)
    observation_schema['value_integer'] = getattr(obs, 'valueInteger', None)
    observation_schema['value_unit'] = getattr(getattr(obs,'valueQuantity',None),'unit',None)
    observation_schema['value'] = getattr(getattr(obs,'valueQuantity',None),'value',None)
    observation_schema['value_tring'] = getattr(obs, 'valueString', None)
    observation_schema['data_absent_reason'] = getattr(obs, 'dataAbsentReason', None)
    observation_schema['interpretation'] = getattr(obs, 'interpretation', None)
    observation_schema['note'] = getattr(obs, 'note', None)
    observation_schema['bodysite'] = getattr(obs, 'bodySite', None)
    observation_schema['method'] = getattr(obs, 'method', None)
    observation_schema['specimen'] = getattr(obs, 'specimen', None)
    observation_schema['device'] = getattr(obs, 'device', None)
    observation_schema['reference_range'] = getattr(obs, 'referenceRange', None)
    observation_schema['has_member'] = getattr(obs, 'hasMember', None)
    observation_schema['derived_from'] = getattr(obs, 'derivedFrom', None)
    observation_schema['component'] = getattr(obs, 'component', None)

    return observation_schema    

To fix:
- Observation Component
- value_codable_concept